# Projeto 2 - Classificando Automático de Sentimento 

## Por: Beatriz Alexandre e Vitória Almeida

#### Marca escolhida: Lindt

Fomos contratadas pela empresa Lindt, pára analisar os tweets feitos por seus clientessobre sua empresa. Para isso criaremos um programa que irá analisae as mensagens disponiveis como "relevante" ou "irrelevante". Os twers que forem negativos irão disparar um foco na área de marketing.

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl  
import seaborn as sns
import numpy as np
import itertools
from scipy import stats
from emoji import UNICODE_EMOJI

#### Colocando os dados em um DataFrame

In [2]:
tw = pd.read_excel('tweets_lindt.xlsx')

In [3]:
tw.head()

,Treinamento,Relevante,Positivo/Negativo
0,lindt chocolate is so overrated,1,0
1,@elgliko hear that pitter patter in the back g...,0,NaN
2,rt @statutory_boy: the guy was addicted to dru...,0,NaN
3,"@andreyasasylum @yourmomsuckstho hey, can you ...",1,1
4,great job by @noticgroup gms. come and see the...,1,1


#### Limpando os tweets

LIMPAR TWEETS

In [4]:
tw.Treinamento = tw.Treinamento.str.replace('@', '')
tw.Treinamento = tw.Treinamento.str.replace(':', '')
tw.Treinamento = tw.Treinamento.str.replace('_', '')
tw.Treinamento = tw.Treinamento.str.replace('%', '')
tw.Treinamento = tw.Treinamento.str.replace('&', '')
tw.Treinamento = tw.Treinamento.str.replace(',', '')
tw.Treinamento = tw.Treinamento.str.replace('#', '')
tw.Treinamento = tw.Treinamento.str.replace(';', '')
tw.Treinamento = tw.Treinamento.str.replace('.', '')
tw.Treinamento = tw.Treinamento.str.replace('$', '')
tw.Treinamento = tw.Treinamento.str.replace('(', '')
tw.Treinamento = tw.Treinamento.str.replace(')', '')
tw.Treinamento = tw.Treinamento.str.replace('*', '')
tw.Treinamento = tw.Treinamento.str.replace('/', '')
tw.Treinamento = tw.Treinamento.str.replace('?', '')
tw.Treinamento = tw.Treinamento.str.replace('+', '')
tw.Treinamento = tw.Treinamento.str.replace('-', '')
tw.Treinamento = tw.Treinamento.str.replace(']', '')
tw.Treinamento = tw.Treinamento.str.replace('[', '')
tw.Treinamento = tw.Treinamento.str.replace('=', '')
tw.Treinamento = tw.Treinamento.str.replace('^', '')
tw.Treinamento = tw.Treinamento.str.replace('!', '')
tw.Treinamento = tw.Treinamento.str.replace('"', '')

tw.head()

,Treinamento,Relevante,Positivo/Negativo
0,lindt chocolate is so overrated,1,0
1,elgliko hear that pitter patter in the back gr...,0,NaN
2,"rt statutoryboy the guy was addicted to drugs,...",0,NaN
3,"andreyasasylum yourmomsuckstho hey, can you pl...",1,1
4,great job by noticgroup gms. come and see them...,1,1


Dar espaço nos emojis:

In [5]:
tweets_espaco_emoji = []
data = tw["Treinamento"]
for tweet in data:
    t = ''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            t = t+ " " + palavra + " "
        else:
            t += palavra
    tweets_espaco_emoji.append(t)

tw_limpo =  pd.DataFrame()
tw_limpo['Treinamento'] = tweets_espaco_emoji
tw_limpo['Relevante'] = tw['Relevante']

tw_limpo.head()

,Treinamento,Relevante
0,lindt chocolate is so overrated,1
1,elgliko hear that pitter patter in the back gr...,0
2,"rt statutoryboy the guy was addicted to drugs,...",0
3,"andreyasasylum yourmomsuckstho hey, can you pl...",1
4,great job by noticgroup gms. come and see them...,1


Lista de todas as palavras dos tweets:

In [6]:
pala = []

for i in tw_limpo['Treinamento']:
    frase = i.split()
    for p in frase:
        if p not in pala:
            pala.append(p)
    

Contar quantas vezes aparace o 1 e o 0:

In [7]:
#contador
Rel = 0
Irre = 0 

for l in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][l].split(' ')
    for i in linha:
        if tw_limpo["Treinamento"][l] == 1:
            Rel += 1
        else:
            Irre += 1

Dicionários para calcular a frequencia de relevantes e irrelevantes:

In [9]:
#adicionou as palavras no dicionário de frequencia
r_freq = {}
ir_freq = {}

for palavras in pala:
    r_freq[palavras] = 1
    ir_freq[palavras] = 1

for g in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][g].split(' ')
    for v in linha:
        if tw_limpo["Relevante"][v] == 0:
            ir_freq[v] += 1
        else:
            r_freq[v] += 1

KeyError: 'lindt'

A probabilidade de ser relevante da cada palavra

In [11]:
r_prob = {}
ir_prob = {}

#pala é a lista de palavras dos tweets

for palavra in pala:
    r_prob[palavra] = r_freq[palavra]/(len(palavra)+Rel)
    ir_prob[palavra] = ir_freq[palavra]/(len(palavra)+Irre)
    

P(relevante) e P(irrelevante)

In [15]:
irrelevante = 0
relevante = 0 
for m in tw_limpo['Relevante']:
    if m == 0:
        irrelevante += 1
    else:
        relevante += 1 

Pi = irrelevante/len(tw_limpo['Relevante'])
Pr = relevante/len(tw_limpo['Relevante'])
Pi


0.5633333333333334

Teoria de Naive - Bayes

In [ ]:
for

# Mudança de índice - Bia fez

A mudança de iíndice foi realizada para que a primeira coluna do DataFrame fossem os tweets

In [ ]:
tw_idx = tw.set_index('Treinamento')
tw_idx

#### Separando Relevante e Irrelevante

In [ ]:
rel = tw_idx[tw_idx.Relevante == 1]
rel

In [ ]:
irrel = tw_idx[tw_idx.Relevante == 0]
irrel

In [ ]:
lista = tw.Treinamento.tolist()
print (lista)

#### Criando as listas

In [ ]:
lista = tw.groupby('Relevante').apply(lambda grupo: grupo.Treinamento.tolist()).tolist()
lista

In [ ]:
lista_r = []
lista_r.append(lista[1])
lista_r

In [ ]:
lista_ir = []
lista_ir.append(lista[0])
lista_ir

In [ ]:
lista_rel = []
for e in lista_r[0]:
    strings = e.split(' ')
    lista_rel.append(strings)
    
lista_rel

In [ ]:
lista_irrel = []
for i in lista_ir[0]:
    string = i.split(' ')
    lista_irrel.append(string)
    
    
lista_irrel

#### Referências
* To list: https://pt.stackoverflow.com/questions/243921/criar-lista-com-conte%C3%BAdo-de-colunas

* Split: https://www.tutorialspoint.com/python/string_split.htm
* Tutorial